#### DataNatureで使うテキストファイル（タブ区切り）をDataFrameとして処理を行い、csvファイルとして出力する。
#### 仕入ファイル用

In [ ]:
# google drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# google driveのpathとinput、outputのファイル名を指定
input_path = "/content/drive/MyDrive/Purchased_products/txt/"
output_path = "/content/drive/MyDrive/Purchased_products/csv/"
input_file_name = "S2021_2022.txt"
output_file_name = "N_S_2021_2022.csv"

# 出力ファイルの項目を指定
columns_name = ""

In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
# テキストファイル読込
df = pd.read_table(input_path + input_file_name, encoding="cp932")

In [ ]:
df.head()

### 半角￥マークつきの金額部分の数値のみ取り出す
###### https://torisky.com/python%EF%BC%9A%E5%8D%8A%E8%A7%92%EF%BF%A5%E3%83%9E%E3%83%BC%E3%82%AF%E3%81%A4%E3%81%8D%E3%81%AE%E9%87%91%E9%A1%8D%E9%83%A8%E5%88%86%E3%81%AE%E6%95%B0%E5%80%A4%E3%81%AE%E3%81%BF%E5%8F%96%E3%82%8A/

In [ ]:
def mark_num_change(mark):
    """
        半角￥マークつきの金額部分の数値のみ取り出す
              例）販売価格 \824,529 --> 824529
    """
    if len(mark) >= 2:
        if mark[-2] == ".":
            mark = mark[:-2]
    # 「\」と「,」と数字以外の文字を消去する
    after = re.sub(r"\D", "", mark)
    # マイナス対応
    if "-" in mark:
        after = int(after) * (-1)
    else:
        after = int(after)
    return after

In [ ]:
# 列名を退避
columns = df.columns

In [ ]:
columns

Index(['手配No', '仕入先CD', '仕入先名', '仕入月', '仕入日', '担当者CD', '担当者名', '部署CD', '部署名',
       '勘定科目CD', '勘定科目名', '部門名', '伝票区分', '品目', '品名', '規格', '大分類CD', '大分類名',
       '分類2', '仕入数', '仕入単価', '仕入金額', '開発経費管理No', '備考', '財務部署', '購買品重要度',
       '国コード', '納期', '軽減税率対象', '医療機器対象品目', '単位'],
      dtype='object')

In [ ]:
# nan対応
df = df.fillna(0)

In [ ]:
# DataFrameをnp.arrayに変換　処理スピードを上げるため
df = np.array(df)

In [ ]:
# 仕入数、仕入単価、仕入金額から￥などの文字を除いて数値にする
columns = list(columns)
a = columns.index("仕入数")
b = columns.index("仕入単価")
c = columns.index("仕入金額")
for i in range(len(df)):
    df[i, a] = mark_num_change(str(df[i, a]))
    df[i, b] = mark_num_change(str(df[i, b]))
    df[i, c] = mark_num_change(str(df[i, c]))

In [ ]:
# 年度データ作成
nendo = np.array([[]])
for i in range(len(df)):
    if int(str(df[i, 3])[-2:]) >= 6:
        nendo = np.append(nendo, int(str(df[i, 3])[:4]))
    else:
        nendo = np.append(nendo, int(str(df[i, 3])[:4]) - 1)

nendo = nendo.reshape(1, len(nendo))
df = np.append(df.T, nendo, axis=0).T

In [ ]:
# DataFrameに戻す
columns.append("年度")
df = pd.DataFrame(df, columns=columns)

In [ ]:
# タイプをIntに
df["仕入数"] = df["仕入数"].astype(int)
df["仕入単価"] = df["仕入単価"].astype(int)
df["仕入金額"] = df["仕入金額"].astype(int)
df["品目"] = df["品目"].astype(int)

In [ ]:
# ファイル出力
df.to_csv(output_path + output_file_name,  encoding="utf_8_sig")